In [4]:
import pandas as pd
import numpy as np
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import xgboost as xgb

df_test = pd.read_csv("test_df_features.csv")



imu_cols_acceleration = [column for column in df_test.columns if column.startswith("acc_")]
imu_cols_rotation = [columns for columns in df_test.columns if columns.startswith("rot_")]
IMU_Cols = imu_cols_acceleration + imu_cols_rotation
Thm_Cols = [c for c in df_test.columns if c.startswith("thm_")]
ToF_Cols = [c for c in df_test.columns if c.startswith("tof_")][:5]
#XGboost_cols = ["subject", "orientation", "behavior", "phase"]

grouped = df_test.groupby("sequence_id")
X_IMU_list = [group[IMU_Cols].values for _, group in grouped]
X_Thm_list = [group[Thm_Cols].values for _, group in grouped]
X_ToF_list = [group[ToF_Cols].values for _, group in grouped]
#X_XGboost_list = [group[XGboost_cols].values for _, group in grouped]
sequence_ids = list(grouped.groups.keys())


X_IMU = pad_sequences(X_IMU_list, maxlen=115, dtype='float32', padding='post', truncating='post')
X_Thm = pad_sequences(X_Thm_list, maxlen=115, dtype='float32', padding='post', truncating='post')
X_ToF = pad_sequences(X_ToF_list, maxlen=64, dtype='float32', padding='post', truncating='post')
#X_XGboost = pad_sequences(X_XGboost_list, dtype='float32', padding='post', truncating='post')
print(f"IMU: {X_IMU.shape} | Thm: {X_Thm.shape} | ToF: {X_ToF.shape} ")


model_imu = load_model("IMU_RNN_LSTM_variables_macrof1.h5")
model_tof = load_model("Tofs_CNN_MacroF1.h5")
model_thm = load_model("THM_RNN_LSTM.h5")

#xgb_model = xgb.XGBClassifier()
#xgb_model.load_model("XGBoost_variables.json")

pred_imu_all = model_imu.predict(X_IMU, verbose=0)
pred_tof_all = model_tof.predict(X_ToF, verbose=0)
pred_thm_all = model_thm.predict(X_Thm, verbose=0)
#pred_xgb_all = xgb_model_predict(X_XGboost)


weighted_pred = 0.4 * pred_imu_all + 0.4 * pred_tof_all + 0.2 * pred_thm_all
weighted_pred = weighted_pred / np.sum(weighted_pred, axis=1, keepdims=True)

pred_indices = np.argmax(weighted_pred, axis=1)


gesture_classes = [
    "Above ear - pull hair",
    "Cheek - pinch skin",
    "Eyebrow - pull hair",
    "Eyelash - pull hair",
    "Forehead - pull hairline",
    "Forehead - scratch",
    "Neck - pinch skin",
    "Neck - scratch",
    "Write name on leg",
    "Wave hello",
    "Glasses on/off",
    "Text on phone",
    "Write name in air",
    "Feel around in tray and pull out an object",
    "Scratch knee/leg skin",
    "Pull air toward your face",
    "Drink from bottle/cup",
    "Pinch knee/leg skin",
    "non_target"
]

pred_labels = [gesture_classes[i] for i in pred_indices]
max_probs = np.max(weighted_pred, axis=1)
results = pd.DataFrame({
    "sequence_id": sequence_ids,
    "pred_label": pred_labels,
    "confidence": max_probs
})
print(results.head())



IMU: (2, 115, 10) | Thm: (2, 115, 5) | ToF: (2, 64, 5) 


  sequence_id         pred_label  confidence
0  SEQ_000001  Write name on leg    0.189136
1  SEQ_000011     Neck - scratch    0.234653
